In [1]:
from utilities.proj1_helpers import load_csv_data

In [2]:
train_data = load_csv_data("data/train.csv")
y, x, ids = train_data[0], train_data[1], train_data[2]
N, D = x.shape
print(y.shape, x.shape, ids.shape)

(250000,) (250000, 30) (250000,)


In [3]:
test_data = load_csv_data("data/test.csv")
y_train, x_train, ids_train = train_data[0], train_data[1], train_data[2]

In [36]:
from regression.linear_models import *
from regression.stochastic_models import *
from utilities.cross_validation import cross_validation, build_k_indices, split_data
from regression.loss import compute_rmse_loss
from utilities.pca import compute_pca
from utilities.stochastic import *
from utilities.preprocessing import standard_scaler
from utilities.proj1_helpers import *

In [5]:
# Params to bruteforce
lambdas = np.logspace(-5, 1, 10)
degrees = np.arange(1, 8, 1)
k_folds = 4
seed = 1

In [16]:
x = standard_scaler(x)
x_reduced, vectors = compute_pca(x)
x_train = standard_scaler(x)
g = np.mean(x_train, axis=0)
x_train_reduced = x_train.dot(vectors) + np.ones((N, 1)).dot(g[:vectors.shape[1]].reshape((1, vectors.shape[1])))
indices = np.random.randint(0, high=len(x), size=((1, 250000)))
indices_rouge = np.where(y[indices] == -1.)[1]
indices_bleu = np.where(y[indices] == 1.)[1]

In [17]:
x_reduced.shape

(250000, 14)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(x_reduced[indices_rouge, 0], x_reduced[indices_rouge, 0], x_reduced[indices_rouge, 2], color="red")
ax.scatter3D(x_reduced[indices_bleu, 0], x_reduced[indices_bleu, 0], x_reduced[indices_bleu, 2], color="blue")
plt.show()

In [ ]:
def generate_w(num_intervals):
    """Generate a grid of values for w0 and w1."""
    w0 = np.linspace(-100, 200, num_intervals)
    w1 = np.linspace(-100, 200, num_intervals)
    w2 = np.linspace(-100, 200, num_intervals)
    return w0, w1, w2


def get_best_parameters(w0, w1, w2, losses):
    """Get the best w from the result of grid search."""
    min_row, min_col, min_slice = np.unravel_index(np.argmin(losses), losses.shape)
    return losses[min_row, min_col, min_slice], w0[min_row], w1[min_col], w2[min_slice]

def grid_search(y, tx, w0, w1, w2):
    """Algorithm for grid search."""
    losses = np.zeros((len(w0), len(w1), len(w2)))
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: compute loss for each combination of w0, w1 and w2.
    # ***************************************************
    for i in range(0, len(w0)):
        for j in range(0, len(w1)):
            for k in range(0, len(w2)):
                losses[i, j, k] = compute_rmse_loss(y, tx, np.array([w0[i], w1[j], w2[k]]))
    return losses

w0, w1, w2 =  generate_w(30)
losses = grid_search(y, x_reduced, w0, w1, w2)
print(get_best_parameters(w0, w1, w2, losses))

In [31]:
best_loss = 1e100
best_params = (-1, -1)
k_indices = build_k_indices(y, k_folds, seed)
degree = 1

D = x_reduced.shape[1]
w0 = np.array([0 for i in range(D)])

for l in lambdas:
    w, loss_tr = stochastic_gradient_descent(y, x_reduced, w0, 100, 1e-1, compute_rmse_loss, compute_mse_gradient)
    loss_te = compute_rmse_loss(y_train, x_train_reduced, w, lambda_=l)
    if loss_te < best_loss:
        best_loss = loss_te
        best_params = (l, degree)

    print("l={}, d={}: train {}, test {}".format(l, degree, loss_tr, loss_te))

Stochastic Gradient Descent(0/999): loss=1.0, w0=-0.06133128229328736, w1=0.0037887671336430786
Stochastic Gradient Descent(1/999): loss=0.9634378932122966, w0=-0.04742584220215018, w1=0.006680059020824096
Stochastic Gradient Descent(2/999): loss=0.9485059737829216, w0=-0.05057857693878169, w1=0.008886467723710116
Stochastic Gradient Descent(3/999): loss=0.9392561047400964, w0=-0.04986376775227502, w1=0.010570226940742214
Stochastic Gradient Descent(4/999): loss=0.9331888156476676, w0=-0.05002583409016815, w1=0.0118551407906705
Stochastic Gradient Descent(5/999): loss=0.9291297137846826, w0=-0.04998908932195041, w1=0.01283568693113807
Stochastic Gradient Descent(6/999): loss=0.9263611685731596, w0=-0.04999742034249994, w1=0.013583963349545036
Stochastic Gradient Descent(7/999): loss=0.9244323597524718, w0=-0.04999553147758765, w1=0.014154989612599804
Stochastic Gradient Descent(8/999): loss=0.9230580133387525, w0=-0.04999595973372214, w1=0.014590752366947287
Stochastic Gradient Descent

In [33]:
print(best_params, best_loss)

(1.0000000000000001e-05, 1) 0.918198581056


In [41]:
submit_data = load_csv_data("data/sample-submission.csv")
y, x, ids = submit_data[0], submit_data[1], submit_data[2]
N, D = x.shape
x = standard_scaler(x)
g = np.mean(x, axis=0)
x = x.dot(vectors) + np.ones((N, 1)).dot(g[:vectors.shape[1]].reshape((1, vectors.shape[1])))
D = x.shape[1]
w0 = np.array([0 for i in range(D)])
w, _ = stochastic_gradient_descent(y, x, w0, 100, 1e-1, compute_rmse_loss, compute_mse_gradient)

ValueError: shapes (568238,0) and (30,14) not aligned: 0 (dim 1) != 30 (dim 0)

In [ ]:
def sums_(n_numbers, d):
    if n_numbers <= 1:
        result = d
    else:
        result = np.zeros((0, n_numbers))
        for i in range(d, -1, -1):
            rc = sums_(n_numbers - 1, d - i)
            print(rc, result.shape)
            result = np.vstack((result, np.hstack((i * rc, rc))))
    return result

X = x_reduced
n_obs, n_vars = X.shape
max_degree  = 4     # order of polynomial

stacked = np.zeros((0, n_vars)) #this will collect all the coefficients...    
for d in range(1, max_degree+1):
    stacked = np.vstack((stacked, sums_(n_vars, d)))

newX = np.zeros((X.shape[0], stacked.shape[0]))
for i in range(1, stacked.shape[0]+1):
    accumulator = np.ones((n_obs, 1))
    for j in range(1, n_vars+1):
        accumulator = accumulator * X[:, j-1]**stacked[i-1, j-1]
    newX[:, i-1] = accumulator

                                
                                
                